In [1]:
import pandas as pd
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import seaborn as sns 
sns.set()
import os
from os import listdir
from os.path import isfile, join
import csv
import numpy as np
%matplotlib inline
from random import sample
from math import sin, cos, sqrt, atan2, radians, acos
import numpy as np
import math
import copy
import time
from datetime import datetime, timedelta



In [2]:
#stop_times table only weekday schedules
#arrival_time_fix and departure_time_fix are the times in seconds 
weekday_stop_time_fixed =  pd.read_csv(r'weekday_stop_time_fixed.csv', sep=",", engine='python')

# route_short_name == the bus number
# the trip_id can be linked with the trip id in stop times
# trip id is unique to each row. 
trips =  pd.read_csv(r'C:\\Users\\이진우\\cs_summer\\Jin_project\\mmt_gtfs\\trips.txt', sep=",", engine='python')


In [34]:
#stop_times table only weekday schedules
#arrival_time_fix and departure_time_fix are the times in seconds 
trip_table_fixed =  pd.read_csv(r'trip_table_fixed.csv', sep=",", engine='python')

In [35]:
#select only the 2 hour time frame of 7-9 am from trip_table_fixed
trip_table_fixed = trip_table_fixed[trip_table_fixed['arrival_time_fix'].between(25200, 32400, inclusive=True)]

In [36]:
#create graph
graph = {}
unique_table=trip_table_fixed.groupby(['full_bus_name','trip_id', 'stop_id_time']).size().reset_index()
for index, row in unique_table.iterrows(): 
    bus_name = unique_table.loc[index, 'full_bus_name']
    graph[bus_name] = {}
for index, row in unique_table.iterrows(): 
    bus_name = unique_table.loc[index, 'full_bus_name']
    bus_trip_id = unique_table.loc[index, 'trip_id']
    stop_id = unique_table.loc[index, 'stop_id_time']    
    graph[bus_name][bus_trip_id] = set([])

    


In [37]:
graph_ready = copy.deepcopy(graph)

In [38]:
#create graph form 
#bus#:set([bus_stops])
for index, row in trip_table_fixed.iterrows():
    route_name = row['full_bus_name']
    stop_id = row['stop_id_time']
    trip_id = row['trip_id']
    try:
        graph_ready[route_name][trip_id].add(stop_id)
    except:
        print(route_name, stop_id, trip_id)
        break

In [39]:
#since the graph is pre ordered in time, no need to put in any time conditions in the loop!
for key in graph_ready:
    for trip_id in graph_ready[key]:                   
        graph_temp = sorted(list(graph_ready[key][trip_id]), key=lambda x: int(x.split('_')[1]))
        graph_ready[key][trip_id] = graph_temp


In [40]:
for key in graph_ready:
    if 'DROP OFF' in key:
        print(key)

10_DROP OFFS ONLY
11_DROP OFFS ONLY
12_DROP OFFS ONLY
13_DROP OFFS ONLY
14_DROP OFFS ONLY
15_DROP OFFS ONLY
16_DROP OFFS ONLY
17_DROP OFFS ONLY
18_DROP OFFS ONLY
19_DROP OFFS ONLY
20_DROP OFFS ONLY
21_DROP OFFS ONLY
22_DROP OFFS ONLY
23_DROP OFFS ONLY
25_DROP OFFS ONLY
26_DROP OFFS ONLY
27_DROP OFFS ONLY
28_DROP OFFS ONLY
29_DROP OFFS ONLY
2_DROP OFFS ONLY
30_DROP OFFS ONLY
31_DROP OFFS ONLY
33_DROP OFFS ONLY
34_DROP OFFS ONLY
35_DROP OFFS ONLY
36_DROP OFFS ONLY
37_DROP OFFS ONLY
38_DROP OFFS ONLY
39_DROP OFFS ONLY
3_DROP OFFS ONLY
40_DROP OFFS ONLY
44_DROP OFFS ONLY
47_DROP OFFS ONLY
48_DROP OFFS ONLY
49_DROP OFFS ONLY
4_DROP OFFS ONLY
50_DROP OFFS ONLY
52_DROP OFFS ONLY
55_DROP OFFS ONLY
56_DROP OFFS ONLY
57_DROP OFFS ONLY
58_DROP OFFS ONLY
5_DROP OFFS ONLY
67_DROP OFFS ONLY
6_DROP OFFS ONLY
71_DROP OFFS ONLY
72_DROP OFFS ONLY
73_DROP OFFS ONLY
75_DROP OFFS ONLY
80_DROP OFFS ONLY


In [41]:
#star_stop will be a stop with a time in the morning between 7-8 AM 
def dfs(graph, start_stop):
     #bus_key = full_bus_name
    stop = start_stop
    visited, stack = set(), [stop] #creates the empty list and empty stack to go throu the while loop
    
    while stack:
        vertex = stack.pop() #takes out one of the values from stack
        current_time = int(vertex.split('_')[1]) #current time of the stop hit
        
        #if visited list is empty, this must be the first hit. 
        #Get all the stops that has the stop hits.
        #add the next stops to the stack. 
        if not visited:
            begin_time = int(stop.split('_')[1])
            visited.add(vertex)
            for bus_name in graph:
#                 #buses that drops off only cannot be a starting point
#                 if 'DROP OFF' in bus_name:
#                     continue
                for trip_id in graph[bus_name]:
                    try:
                        index_of_stop = graph[bus_name][trip_id].index(vertex)
                        next_stop = graph[bus_name][trip_id][index_of_stop+1]
                        stack.extend([next_stop])
                    except: 
                        continue
        else:          
            #checks if the time has passed 1 hour.
            #if it is more than 1 hour, simply continue to the next node. 
            if current_time - begin_time < 3600:
                if vertex not in visited:
                    visited.add(vertex)
                    for bus_name in graph:
                        for trip_id in graph[bus_name]:
#                             if 'DROP OFF' in bus_name:
#                                 try:
#                                     #check if the previous stop was the same trip id.
#                                     #if it is, on the same bus so drop off only is fine
#                                     stop_id=drop_off_only[trip_id]
#                                     if stop_id == vertex:
#                                         pass
#                                     else: 
#                                         continue
#                                 except: 
#                                     continue
                            try:
                                index_of_stop = graph[bus_name][trip_id].index(vertex)
                                next_stop = graph[bus_name][trip_id][index_of_stop+1]
                                stack.extend([next_stop])
#                                 if 'DROP OFF' in bus_name:
#                                     drop_off_only[trip_id] = vertex
                            except: 
                                continue
            else: 
                continue 
    return visited



In [42]:
list_of_stops = trip_table_fixed['stop_id_time'].unique()
#since the graph is pre ordered in time, no need to put in any time conditions in the loop!
list_of_stops_fixed = sorted(list_of_stops, key=lambda x: int(x.split('_')[1]))


In [45]:
dic_of_rows = {}
for start_stop in list_of_stops_fixed:
    dic_of_rows[start_stop]=[]
count = 0
#choose a stop to first begin
for start_stop in list_of_stops_fixed:
    list_of_stop=dfs(graph_ready, start_stop)
    dic_of_rows[start_stop].append(list_of_stop)
    print('starting stop is ', start_stop)
#     if count == 5:
#         break
#     count += 1
    



starting stop is  6100_25200
starting stop is  5100_25200
starting stop is  1878_25200
starting stop is  4100_25200
starting stop is  2200_25200
starting stop is  3196_25200
starting stop is  1101_25200
starting stop is  1486_25200
starting stop is  6350_25200
starting stop is  6476_25200
starting stop is  4401_25200
starting stop is  1100_25200
starting stop is  10_25200
starting stop is  2195_25200
starting stop is  9800_25200
starting stop is  3841_25201
starting stop is  7991_25201
starting stop is  1410_25202
starting stop is  435_25202
starting stop is  2923_25203
starting stop is  435_25204
starting stop is  9996_25204
starting stop is  758_25204
starting stop is  2885_25205
starting stop is  445_25206
starting stop is  6539_25206
starting stop is  8869_25206
starting stop is  2146_25207
starting stop is  1579_25207
starting stop is  5236_25207
starting stop is  8869_25207
starting stop is  5715_25208
starting stop is  1968_25208
starting stop is  7723_25208
starting stop is  18

starting stop is  8297_25320
starting stop is  10_25320
starting stop is  5100_25320
starting stop is  133_25320
starting stop is  1505_25320
starting stop is  3301_25320
starting stop is  668_25323
starting stop is  2270_25323
starting stop is  839_25323
starting stop is  8333_25323
starting stop is  8333_25324
starting stop is  2091_25325
starting stop is  499_25327
starting stop is  9192_25327
starting stop is  1579_25327
starting stop is  688_25327
starting stop is  971_25328
starting stop is  703_25329
starting stop is  9984_25330
starting stop is  1462_25330
starting stop is  2620_25331
starting stop is  1462_25331
starting stop is  2902_25332
starting stop is  2161_25332
starting stop is  8418_25332
starting stop is  4192_25332
starting stop is  2401_25332
starting stop is  1309_25332
starting stop is  2323_25333
starting stop is  941_25333
starting stop is  7840_25333
starting stop is  9154_25333
starting stop is  9725_25334
starting stop is  9154_25334
starting stop is  2383_2

starting stop is  6306_25449
starting stop is  9313_25449
starting stop is  9313_25450
starting stop is  3368_25450
starting stop is  1968_25451
starting stop is  1480_25451
starting stop is  2698_25453
starting stop is  1256_25453
starting stop is  839_25453
starting stop is  388_25455
starting stop is  1213_25455
starting stop is  5109_25456
starting stop is  568_25456
starting stop is  9182_25457
starting stop is  3946_25457
starting stop is  9783_25457
starting stop is  5823_25457
starting stop is  4845_25458
starting stop is  2643_25458
starting stop is  5320_25459
starting stop is  1959_25460
starting stop is  9493_25460
starting stop is  6482_25460
starting stop is  9163_25460
starting stop is  9750_25461
starting stop is  692_25461
starting stop is  5823_25461
starting stop is  1435_25462
starting stop is  7379_25462
starting stop is  1482_25462
starting stop is  184_25462
starting stop is  2026_25462
starting stop is  1482_25463
starting stop is  9356_25463
starting stop is  2

starting stop is  915_25575
starting stop is  5346_25575
starting stop is  1901_25575
starting stop is  2266_25576
starting stop is  454_25576
starting stop is  6516_25576
starting stop is  9551_25577
starting stop is  4623_25577
starting stop is  6848_25577
starting stop is  298_25577
starting stop is  1805_25579
starting stop is  1112_25579
starting stop is  843_25579
starting stop is  379_25580
starting stop is  9880_25580
starting stop is  5_25580
starting stop is  9744_25580
starting stop is  1112_25580
starting stop is  2191_25581
starting stop is  6704_25581
starting stop is  4456_25582
starting stop is  184_25582
starting stop is  9140_25583
starting stop is  8712_25584
starting stop is  2793_25584
starting stop is  2712_25584
starting stop is  391_25585
starting stop is  1785_25585
starting stop is  234_25585
starting stop is  3980_25586
starting stop is  9552_25587
starting stop is  2678_25587
starting stop is  234_25587
starting stop is  7209_25587
starting stop is  1790_255

starting stop is  6818_25697
starting stop is  9532_25698
starting stop is  2522_25699
starting stop is  2752_25699
starting stop is  4957_25700
starting stop is  5796_25700
starting stop is  6785_25700
starting stop is  1936_25701
starting stop is  2195_25701
starting stop is  6418_25701
starting stop is  455_25702
starting stop is  1156_25702
starting stop is  941_25702
starting stop is  1834_25703
starting stop is  8153_25703
starting stop is  1901_25704
starting stop is  5482_25705
starting stop is  7846_25705
starting stop is  6812_25706
starting stop is  392_25707
starting stop is  3518_25707
starting stop is  512_25708
starting stop is  7758_25708
starting stop is  904_25709
starting stop is  2665_25709
starting stop is  4714_25710
starting stop is  843_25710
starting stop is  2197_25710
starting stop is  706_25710
starting stop is  4446_25711
starting stop is  1901_25711
starting stop is  203_25711
starting stop is  2465_25711
starting stop is  2267_25711
starting stop is  1105

...

starting stop is  7560_31740
starting stop is  7925_31740
starting stop is  336_31740
starting stop is  2145_31740
starting stop is  568_31740
starting stop is  1784_31740
starting stop is  2344_31741
starting stop is  1482_31741
starting stop is  4655_31742
starting stop is  9264_31742
starting stop is  2984_31743
starting stop is  7417_31743
starting stop is  7115_31744
starting stop is  2201_31744
starting stop is  6357_31746
starting stop is  6527_31746
starting stop is  2916_31746
starting stop is  6627_31747
starting stop is  480_31747
starting stop is  2023_31747
starting stop is  1169_31747
starting stop is  2669_31748
starting stop is  9135_31748
starting stop is  327_31748
starting stop is  8427_31748
starting stop is  2191_31748
starting stop is  4523_31748
starting stop is  2620_31748
starting stop is  2253_31748
starting stop is  6342_31749
starting stop is  2088_31749
starting stop is  8427_31749
starting stop is  9849_31751
starting stop is  730_31752
starting stop is  4

starting stop is  2175_31860
starting stop is  568_31860
starting stop is  7969_31860
starting stop is  10_31860
starting stop is  4100_31860
starting stop is  100_31860
starting stop is  6925_31861
starting stop is  581_31862
starting stop is  1628_31863
starting stop is  1304_31864
starting stop is  2620_31864
starting stop is  2274_31865
starting stop is  883_31865
starting stop is  1831_31865
starting stop is  9348_31867
starting stop is  1928_31868
starting stop is  9977_31870
starting stop is  2370_31870
starting stop is  6447_31870
starting stop is  5856_31871
starting stop is  809_31871
starting stop is  9439_31872
starting stop is  4466_31873
starting stop is  9264_31873
starting stop is  839_31873
starting stop is  6291_31874
starting stop is  4980_31875
starting stop is  568_31876
starting stop is  6516_31876
starting stop is  8241_31877
starting stop is  6489_31877
starting stop is  9677_31878
starting stop is  234_31878
starting stop is  1447_31878
starting stop is  8241_3

starting stop is  2562_31980
starting stop is  6365_31980
starting stop is  2175_31980
starting stop is  7235_31980
starting stop is  1101_31980
starting stop is  2561_31980
starting stop is  184_31980
starting stop is  160_31980
starting stop is  1682_31980
starting stop is  5100_31980
starting stop is  9800_31980
starting stop is  757_31981
starting stop is  2101_31982
starting stop is  8605_31982
starting stop is  2923_31982
starting stop is  353_31982
starting stop is  2921_31983
starting stop is  1618_31984
starting stop is  4837_31984
starting stop is  6860_31984
starting stop is  7520_31985
starting stop is  532_31985
starting stop is  2014_31985
starting stop is  2752_31985
starting stop is  1750_31986
starting stop is  6633_31986
starting stop is  568_31987
starting stop is  1579_31987
starting stop is  2105_31987
starting stop is  1211_31988
starting stop is  6265_31989
starting stop is  1127_31990
starting stop is  488_31990
starting stop is  1480_31991
starting stop is  883

starting stop is  4188_32105
starting stop is  2397_32105
starting stop is  7734_32105
starting stop is  2901_32106
starting stop is  9291_32108
starting stop is  1905_32108
starting stop is  965_32108
starting stop is  2996_32109
starting stop is  9928_32110
starting stop is  7778_32110
starting stop is  1380_32111
starting stop is  6984_32112
starting stop is  1712_32113
starting stop is  173_32113
starting stop is  941_32113
starting stop is  6979_32113
starting stop is  2501_32114
starting stop is  7640_32114
starting stop is  6987_32114
starting stop is  2855_32115
starting stop is  4191_32116
starting stop is  7199_32116
starting stop is  2978_32116
starting stop is  2125_32117
starting stop is  6987_32118
starting stop is  2996_32119
starting stop is  2228_32119
starting stop is  488_32120
starting stop is  1645_32121
starting stop is  9153_32121
starting stop is  568_32122
starting stop is  2230_32122
starting stop is  1412_32122
starting stop is  2881_32122
starting stop is  2

starting stop is  1131_32245
starting stop is  1513_32246
starting stop is  6533_32247
starting stop is  9463_32247
starting stop is  1513_32247
starting stop is  9463_32248
starting stop is  2192_32248
starting stop is  1961_32248
starting stop is  1579_32248
starting stop is  2883_32250
starting stop is  9551_32250
starting stop is  2741_32251
starting stop is  6327_32252
starting stop is  1843_32252
starting stop is  757_32252
starting stop is  883_32253
starting stop is  2497_32253
starting stop is  2349_32254
starting stop is  581_32254
starting stop is  564_32254
starting stop is  1961_32255
starting stop is  9198_32255
starting stop is  7182_32258
starting stop is  8829_32258
starting stop is  133_32258
starting stop is  688_32259
starting stop is  4707_32259
starting stop is  2923_32259
starting stop is  2054_32259
starting stop is  7341_32259
starting stop is  1480_32260
starting stop is  6482_32262
starting stop is  6338_32262
starting stop is  809_32263
starting stop is  215

In [46]:
dic_of_rows

{'6100_25200': [{'1101_27360',
   '1121_27602',
   '1121_27642',
   '1151_27647',
   '1151_27694',
   '1177_27897',
   '1177_27957',
   '1209_27720',
   '1209_27780',
   '1261_27484',
   '1261_27504',
   '1267_27857',
   '1324_27286',
   '1324_27292',
   '1325_27771',
   '1325_27831',
   '133_26640',
   '133_26700',
   '1379_27960',
   '1379_28020',
   '1397_27578',
   '1397_27614',
   '1495_27433',
   '1495_27445',
   '1595_27814',
   '1707_27850',
   '1707_27910',
   '1787_27403',
   '1787_27410',
   '1831_27543',
   '1831_27573',
   '2101_25622',
   '2101_25682',
   '2111_25548',
   '2111_25549',
   '2121_25499',
   '2121_25549',
   '2131_26160',
   '2131_26220',
   '2160_25350',
   '2161_25332',
   '2161_25434',
   '2161_25435',
   '2175_25560',
   '2175_25620',
   '2191_25872',
   '2191_25932',
   '2201_25444',
   '2201_25485',
   '2233_25591',
   '2233_25592',
   '2249_25375',
   '2249_25405',
   '2323_25275',
   '2323_25333',
   '2323_25334',
   '2324_25246',
   '2349_26304',
  

In [50]:
with open('DFS_output_7-9AM.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in dic_of_rows.items():
       writer.writerow([key, value])
csv_file.close() 

In [55]:
with open('DFS_output_7-9AM.csv') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        row=row
        break
row

['6100_25200',
 "[{'2160_25350', '6154_25336', '6418_25701', '1261_27484', '2561_26449', '2615_25486', '8100_25669', '2925_25270', '2741_25786', '4368_25615', '8650_26100', '6350_26100', '2615_25487', '6750_26520', '2945_26061', '6427_26389', '2615_25361', '6511_26415', '6733_26666', '2741_25846', '2131_26220', '2752_25699', '2101_25622', '4580_25811', '2752_25759', '6926_25848', '4663_25986', '8712_25584', '6266_25442', '2249_25375', '4342_25515', '8418_25332', '2889_25680', '2401_25332', '4539_26206', '2595_25973', '4685_26194', '8118_25474', '941_27065', '2945_26121', '8166_25841', '1151_27647', '6426_25756', '2755_25929', '2916_26364', '8301_25912', '6296_25272', '839_27166', '4202_25778', '4761_26160', '1121_27602', '2111_25548', '6787_25984', '1325_27831', '6720_25765', '6605_26301', '6330_25635', '8237_26290', '1177_27897', '2349_26332', '2755_25989', '1707_27910', '2939_25240', '8427_26349', '839_27149', '1787_27410', '2925_25262', '8531_26209', '6923_26079', '1325_27771', '139